<a href="https://colab.research.google.com/github/SaiRajesh228/DA6401_Assignment3/blob/main/DA6401A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir -p /content/transliteration_project
%cd /content/transliteration_project

/content/transliteration_project


In [ ]:

!pip install pandas
!npm install numpy torch matplotlib wandb


⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙npm warn deprecated torch@0.2.7: Package no longer supported. Contact Support at https://www.npmjs.com/support for more info.
⠙
added 11 packages in 2s
⠹

In [ ]:
import wandb
# Login to WandB - this will generate a link to authenticate
wandb.login()

<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: cs24m040 (cs24m040-iit-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
%%writefile Core_Utils.py

import numpy as np
import random
import pandas as pd
import gc

import torch
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence

seed = 23

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

class LanguageProcessor:

    def __init__(self,language_directory,target_lang_name,mode="train",meta_tokens=True):

        """
        Default Constructor for this class.

        Params:

            language_directory : ex : "aksharantar_sampled/tel/"
            mode : "train" or "test" or "valid", accordingly the appropriate dataset is read.
            meta_tokens : If true creates the first three tokens of the dictionary as <start>,<end>,<pad>.

        """

        self.meta_tokens = meta_tokens ## boolean variable, if 1, then <start>,<end> and <pad> tokens are cosidered in the vocab.
        self.language_directory = language_directory
        self.target_lang_name = target_lang_name
        self.mode = mode ## accordingly helps to read and generate the appropriate dataset.

        self.source_lang = 0
        self.target_lang = 1

        self.source_char2id,self.source_id2char = self.build_char_vocab(self.source_lang)
        self.target_char2id,self.target_id2char = self.build_char_vocab(self.target_lang)


    def build_char_vocab(self,lang_id,max_len=None):

        """
        Method to create a vocabulary of characters in language corresponding to lang_id.

        lang_id : 0/1 indicating the source and target language respectively.
        max_len : Depricated!

        Returns:
            None.
        """

        # Modified to ensure all data is read as strings and handle tab separators
        df = pd.read_csv(self.language_directory+self.mode+".txt",
                         header=None,
                         sep='\t',
                         dtype=str,  # Enforce string type for all columns
                         na_filter=False)  # Don't interpret empty fields as NaN

        # Only keep the first two columns for source and target languages
        if df.shape[1] > 2:
            df = df.iloc[:, 0:2]

        self.data = df.to_numpy()

        lang_chars = []
        lang_words = df[lang_id].to_numpy()

        for word in lang_words:
            # Make sure word is treated as a string
            if word is not None and word != '':
                lang_chars += list(str(word))

        unique_lang_chars = sorted(list(set(lang_chars)))

        if self.meta_tokens:
            char2id_dict = {'<start>':0,'<end>':1,'<pad>': 2}
            id2char_dict = {0:'<start>',1:'<end>',2:'<pad>'}

            ## These ids would still hold irrespective of the target language
            self.start_token_id = char2id_dict['<start>']
            self.end_token_id = char2id_dict['<end>']
            self.pad_token_id = char2id_dict['<pad>']

        else:
            char2id_dict = {}
            id2char_dict = {}

        start = len(char2id_dict) ##Key of each language character starts based on meta tokens are used or not.

        for i in range(len(unique_lang_chars)):
            char2id_dict[unique_lang_chars[i]] = i+start
            id2char_dict[i+start] = unique_lang_chars[i]

        del df
        del lang_chars
        del unique_lang_chars

        gc.collect()

        return char2id_dict,id2char_dict

    def encode_word(self,word,lang_id,padding=False,append_eos = False):

        """
        Method to encode characters of a given word.

        Params:

            word: The word to be encoded.
            lang_id : 0/1 for source/target lang.
            padding [Default - False] : If true, the word encoding would be padded upto max len.
            append_eos [Default - False] : Appends <end> token at the end of every word.

        Returns:

            Encoded Word (np array of character id's)

        """

        if lang_id == self.source_lang:
            char2id_dict = self.source_char2id

        else:
            char2id_dict = self.target_char2id

        word_encoding = []

        # Ensure we're working with a string
        word = str(word).lower()

        for i in word:
            word_encoding.append(char2id_dict[i])

        if append_eos:
            word_encoding.append(char2id_dict['<end>'])

        return np.array(word_encoding)

    def decode_word(self,code_word,lang_id):

        """
        Method to decode an encoded word.

        Params:

            code_word : The encoded word.
            lang_id : 0/1 for source/target lang.
        """

        word = []

        if lang_id == self.source_lang:
            id2char_dict = self.source_id2char
            char2id_dict = self.source_char2id

        else:
            id2char_dict = self.target_id2char
            char2id_dict = self.target_char2id

        start_idx = 0

        for i in code_word[start_idx:]:
            ## if we reached <end>, then stop decoding
            if self.meta_tokens and i == char2id_dict['<end>'] or i == char2id_dict['<pad>']:
                break

            word.append(id2char_dict[i])

        return np.array(word)

class WordDataset(Dataset):

    """
    Class that inherits and overrides the methods of Dataset class. This helps in creating a data loader.
    """

    def __init__(self, language_processor,append_eos=True,device='cpu'):

        """
        The constructor of the class.

        Params:

            language_processor : An object of the class LanguageProcessor, helps to encode and decode words into string of integers.
            append_eos: A boolean param, if True, an <eos> tag is added at the end of the encoded input.
            device : The device on which the processing happens, default : CPU.

        Returns:
            None

        """

        self.lp = language_processor
        self.data = self.lp.data
        self.device = device
        self.append_eos = append_eos
        self.start_token_id = self.lp.start_token_id
        self.end_token_id = self.lp.end_token_id
        self.pad_token_id = self.lp.pad_token_id

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        """
        The method to return the pair (input,target) words, after encoding them

        Params:
            idx : The index corresponding to the pair of words to be returned.

        Returns:
            None

        """
        input_word, output_word = self.data[idx]

        input_sequence = self.lp.encode_word(input_word,self.lp.source_lang,padding=False,append_eos=self.append_eos)
        output_sequence = self.lp.encode_word(output_word,self.lp.target_lang,padding=False,append_eos=self.append_eos)

        return torch.tensor(input_sequence).to(self.device), torch.tensor(output_sequence).to(self.device)

def collate_fn(batch,pad_token_id,device):

    """
    The method to collate on a batch of data, by adding padding based on the longest string in the batch.

    Hence, there is no need to compute a globally longest string and use that size.

    """

    input_words, target_words = zip(*batch)

    padded_inputs = pad_sequence(input_words, batch_first=True, padding_value=pad_token_id)

    padded_targets = pad_sequence(target_words, batch_first=True, padding_value=pad_token_id)

    input_lengths = torch.LongTensor([len(seq) for seq in input_words]).to(device)
    target_lengths = torch.LongTensor([len(seq) for seq in target_words]).to(device)

    return padded_inputs, padded_targets, input_lengths, target_lengths

Writing Core_Utils.py


In [ ]:
%%writefile Encoder_Decoder_Architecture.py

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import random

seed = 23
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

class BahdanauAttention(nn.Module):
    """
    The class to implement Additive attention aka Bhadanau Attention. As seen in lectures.
    """
    def __init__(self, hidden_size,D,expected_dim,batch_size):
        super(BahdanauAttention, self).__init__()
        self.U_att = nn.Linear(hidden_size*expected_dim, hidden_size)
        self.W_att = nn.Linear(hidden_size*D, hidden_size)
        self.V_att = nn.Linear(hidden_size, 1)
        self.batch_size = batch_size

    def forward(self, decoder_prev_hidden, encoder_contexts):

        """
        The method that takes decoder hiddenstates and encoder hidden contexts to produce attention weighted context vector.
        Params:
            decoder_prev_hidden : The decoder's hidden state at t-1.
            encoder_contexts : The encoder hidden states from all time steps.

        """

        decoder_prev_hidden = decoder_prev_hidden.reshape(self.batch_size,1,-1)

        ## Following the same expression as seen in lectures. [Slide 256 in https://www.cse.iitm.ac.in/~miteshk/CS7015/Slides/Teaching/pdf/Lecture16.pdf]
        scores = self.V_att(torch.tanh(self.U_att(decoder_prev_hidden.reshape(self.batch_size,1,-1)) + self.W_att(encoder_contexts))).squeeze(2).unsqueeze(1)
        weights = F.softmax(scores, dim=-1)
        ##compute context of each word in the batch, by considering attention
        context = torch.bmm(weights, encoder_contexts)

        return context, weights


class Encoder(nn.Module):
    """
    The class that implements the encoder using Recurrent Units RNN/LSTM/GRU, as needed, by extending the nn.Module class from torch.

    """
    def __init__(self, source_vocab_size,hidden_size,embedding_size,rnn_type = "GRU",padding_idx = None ,dropout=0.1,num_layers = 1,bidirectional = False):

        """
        The constructor of the Encoder Class.

        Params:

            source_vocab_size : The vocabulary size of the source language.
            hidden_size : The dimension of the hidden state of the recurrent cell.
            embedding_size : The dimension of the embedding used.
            rnn_type : "GRU"/"LSTM"/"RNN", case INsensitive. Default : "GRU".
            padding_idx : The id corresponding to the token <pad>.
            dropout : Droput probability. Default : 0.1
            num_layers(int) : The number of encoder (recurrence unit) layers. Default : 1
            bidirectional : True/False. If True, encoding is done by parsing input L->R and R->L, hence doubling the hiddenstate size. Default False.

        Return:
            None.

        """

        super(Encoder, self).__init__()

        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.embedding_size = self.hidden_size
        self.rnn_type = rnn_type

        self.D = 1 ##the number of directions in which the input is viewed.
        if bidirectional:
            self.D = 2

        self.rnn_dropout = 0
        if self.num_layers>1:
            self.rnn_dropout = dropout

        ##create an embedding layer, and ignore padding index
        self.embedding = nn.Embedding(source_vocab_size, self.embedding_size,padding_idx = padding_idx)

        if self.rnn_type == "GRU":
            self.rnn = nn.GRU(self.embedding_size, self.hidden_size, batch_first=True,num_layers = num_layers,bidirectional = bidirectional,dropout=self.rnn_dropout)
        elif self.rnn_type == "RNN":
            self.rnn = nn.RNN(self.embedding_size, self.hidden_size, batch_first=True,num_layers = num_layers,bidirectional = bidirectional,dropout=self.rnn_dropout)
        elif self.rnn_type == "LSTM":
            self.rnn = nn.LSTM(self.embedding_size, self.hidden_size, batch_first=True,num_layers = num_layers,bidirectional = bidirectional,dropout=self.rnn_dropout)

        self.dropout = nn.Dropout(dropout)

    def forward(self, input,hidden=None,cell=None):

        """
        The method to perform forward pass of the encoder.

        Params :
            Input : The encoded batch of input tensors.
            hidden : Default is None. If the unit is LSTM, it is the previous hidden state.
            cell : Default is None. If the unit is LSTM, it is the cell state.

        Returns :
            output, hidden, cell.
            hidden and cell are current hidden and cell states in case of LSTM and they are None in other cases.

        """

        input_embedding = self.dropout(self.embedding(input))

        if self.rnn_type == "LSTM":
            output, (hidden, cell) = self.rnn(input_embedding, (hidden, cell))
        else:
            output, hidden = self.rnn(input_embedding)
            cell = None

        return output, hidden, cell



class Decoder(nn.Module):
    """
    The class to implement Decoder in the encoder-decoder architecture using "RNN"/"LSTM"/"GRU".

    While the code is flexible enough to support separate types of recurrent units for encoder and decoder,
    In this assignment, I have chosen to use the same type recurrent unit for both.

    """

    def __init__(self, hidden_size,embedding_size,target_vocab_size,rnn_type,batch_size,use_attention = True,padding_idx = None,num_layers = 1,bidirectional = False,dropout=0,device = "cpu"):

        """
        The constructor of this class. Perfoms setup necessary for training.

        hidden_size : The dimension of the hidden state of the recurrent cell.
        embedding_size : The dimension of the embedding used.
        target_vocab_size : The vocabulary size of the target language.
        rnn_type : "GRU"/"LSTM"/"RNN", case INsensitive. Default : "GRU".
        batch_size : The batch size used for training. This is needed to resize dimensions in the BahdanauAttention's forward pass.
        use_attention : Boolean variable, default True, indicating to make use of BahdanauAttention.
        padding_idx : The id corresponding to the token <pad>.
        dropout : Droput probability. Default : 0.1
        num_layers(int) : The number of encoder (recurrence unit) layers. Default : 1
        bidirectional : True/False. If True, encoding is done by parsing input L->R and R->L, hence doubling the hiddenstate size. Default False.
        device : The device on which the processing happens. Default : "cpu".

        """

        super(Decoder, self).__init__()

        self.num_layers = num_layers
        self.rnn_type = rnn_type
        self.device = device
        self.D = 1 ##the number of directions in which the input is viewed.
        self.hidden_size = hidden_size
        self.embedding_size = self.hidden_size
        self.use_attention = use_attention
        if bidirectional:
            self.D = 2

        ## In h0 (the input to the decoder) first dimension expected is number of directions X number of layers
        self.expected_h0_dim1 = self.D*self.num_layers

        ##create an embedding layer, and ignore padding index
        if self.use_attention:
            factor = self.D
        else:
            factor = 1
        self.embedding = nn.Embedding(target_vocab_size, self.embedding_size*factor,padding_idx = padding_idx)

        if self.use_attention:
            self.attention = BahdanauAttention(self.hidden_size,self.D,self.expected_h0_dim1,batch_size)
            recurrent_unit_input_dim = 2*self.D*self.hidden_size
        else:
            recurrent_unit_input_dim = self.embedding_size

        self.rnn_dropout = 0
        if self.num_layers>1:
            self.rnn_dropout = dropout

        #self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True,num_layers = num_layers,bidirectional = bidirectional)
        if self.rnn_type == "GRU":
            self.rnn = nn.GRU(recurrent_unit_input_dim, self.hidden_size, batch_first=True,num_layers = num_layers,bidirectional = bidirectional,dropout=self.rnn_dropout)
        elif self.rnn_type == "RNN":
            self.rnn = nn.RNN(recurrent_unit_input_dim, self.hidden_size, batch_first=True,num_layers = num_layers,bidirectional = bidirectional,dropout=self.rnn_dropout)
        elif self.rnn_type == "LSTM":
            self.rnn = nn.LSTM(recurrent_unit_input_dim, self.hidden_size, batch_first=True,num_layers = num_layers,bidirectional = bidirectional,dropout=self.rnn_dropout)

        ## Passing the hidden state through a fully conencted layer and then applying softmax
        self.output_layer = nn.Linear(self.hidden_size*self.D, target_vocab_size)

    def forward(self, encoder_hidden_contexts, encoder_last_hidden,encoder_cell,target_tensor,eval_mode = False,teacher_forcing_ratio=0):
        """
        Method that Implements the forward pass of the decoder.

        Params:

            encoder_hidden_contexts : The hidden contexts from all time steps of the encoder.
            encoder_last_hidden : The the last hidden state of the encoder, the context, passed as the first decoder_hidden.
            encoder_cell : It is relevant iff the recurrent unit used is LSTM, all other times it would be 0.
            eval_mode : Boolean variable, if true, it adjusts the dimensions to predict for a single word.
            teacher_forcing_ratio : value in [0,1]. It is essentially the probability, with which true input is fed into the decoder at a time step. Default is 0.

            Returns decoder_outputs,decoder_hidden

        """

        batch_size = encoder_hidden_contexts.size(0)
        if not eval_mode:
            max_word_len = target_tensor.size(1)

        ## eval mode is for looking at a specific word that is predicted to compare with the correct word.
        if eval_mode:
            batch_size = 1
            max_word_len = 30 ## an arbitrary number, larger in expecected sense.

        #the decoder predicts one character at a time, and hence we use a list to store all the predictions.
        decoder_outputs = []
        if self.use_attention:
            attentions = []
        else:
            attentions = None

        ## At the first time step <SOS> token (which has an id 0, is fed as an input to the decoder).
        decoder_input = torch.zeros((batch_size, 1), dtype=torch.long, device=self.device)
        decoder_hidden = encoder_last_hidden ## in the first time step of the decoder, the output of the encoder is the input.
        decoder_cell = encoder_cell ## the cell state, which is initially same as that of encoder, (applies to LSTM unit only)

        for step in range(max_word_len):

            ## eval mode is for looking at a specific word that is predicted to compare with the correct word.
            if eval_mode:
                decoder_input = decoder_input.view(1,-1)

            embedding = self.embedding(decoder_input)

            if decoder_hidden.shape[0] != self.expected_h0_dim1:
                reshaped_hidden = decoder_hidden.repeat(self.expected_h0_dim1,1,1)
            else:
                reshaped_hidden = decoder_hidden


            if self.use_attention:
                ## the attention part.
                decoder_prev_hidden = reshaped_hidden.permute(1, 0, 2)
                context_vector, attention_weights = self.attention(decoder_prev_hidden, encoder_hidden_contexts)
                tmp_input = torch.cat((embedding, context_vector), dim=2)
            else:
                ## introducing non-lineartiy through ReLU activation
                activated_embedding = F.relu(embedding)
                tmp_input = activated_embedding


            if self.rnn_type == "LSTM":
                tmp_output, (decoder_hidden, decoder_cell) = self.rnn(tmp_input, (reshaped_hidden, decoder_cell))
            else:
                tmp_output, decoder_hidden = self.rnn(tmp_input, reshaped_hidden)
                cell = None

            decoder_output = self.output_layer(tmp_output.squeeze(0))

            ## randomly sample a number in (0,1) and if the number is less than the teacher forcing ratio
            ## apply teacher forcing at the current step
            apply_teacher_forcing = random.random() < teacher_forcing_ratio

            if (target_tensor is not None) and (apply_teacher_forcing):

                ## Teacher forcing: Feed the target as the next input
                ## extract the 't'th token from th target string to feed as input at "t"th time step.
                decoder_input = target_tensor[:, step].unsqueeze(1) # Teacher forcing
            else:
                ##greedily pick predictions, i.e pick the character corresponding to the hightest probability
                _,preds = torch.max(decoder_output,dim=2)
                decoder_input = preds.detach()

            decoder_outputs.append(decoder_output)
            if self.use_attention:
                attentions.append(attention_weights)

        ## concatenate the predictions across all the timesteps into a singel tensor
        ## found in literature that log_softmax does better than softmax, hence going with that.
        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)

        ## the idea is to have a common API for both attention and normal decoder, achiveing ease of use.
        return decoder_outputs, decoder_hidden,attentions

Writing Encoder_Decoder_Architecture.py


In [ ]:
%%writefile Machine_Translator.py

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import random
from torch import optim
import wandb

from Encoder_Decoder_Architecture import *

from tqdm import tqdm

seed = 23
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

class MachineTranslator:

    """
    The class that instantiates the encoder-decoder architecture and brings all methods relevant for training, computing accuracy and evaluation here.
    """

    def __init__(self,source_vocab_size,target_vocab_size,hidden_size,embedding_size,rnn_type,batch_size,pad_token_id,dropout,num_layers,bidirectional,use_attention,device):

        """
        The constructor of the class.

        Params:

            source_vocab_size : The vocabulary size of the source language.
            target_vocab_size : The vocabulary size of the target language.
            hidden_size : The dimension of the hidden state of the recurrent cell.
            embedding_size : The dimension of the embedding used.
            rnn_type : "GRU"/"LSTM"/"RNN", case INsensitive. Default : "GRU".
            batch_size : The batch size used for training. This is needed to resize dimensions in the BahdanauAttention's forward pass.
            pad_token_id : The id corresponding to the token <pad>.
            dropout : Droput probability. Encoder and Decoder by default use a dropout of 0.1, unless specified otherwise.
            num_layers(int) : The number of encoder (recurrence unit) layers. Default : 1
            bidirectional : True/False. If True, encoding is done by parsing input L->R and R->L, hence doubling the hiddenstate size. Default False.
            use_attention : Boolean variable, default True, indicating to make use of BahdanauAttention.

            Note : hidden_size,embedding_size,dopout, num_layers,bidirectional, rnn_type. These parameters are consistent across the encoder and decoder.
                    However, the code supports use of different values.

        Returns:

            None.


        """
        self.device = device

        self.encoder = Encoder(source_vocab_size = source_vocab_size, hidden_size = hidden_size,embedding_size=embedding_size,rnn_type = rnn_type,padding_idx=pad_token_id,num_layers=num_layers,bidirectional=bidirectional,dropout=dropout).to(self.device)

        self.decoder = Decoder(hidden_size = hidden_size,embedding_size=embedding_size, target_vocab_size = target_vocab_size,batch_size = batch_size,rnn_type = rnn_type,use_attention = use_attention, padding_idx = pad_token_id,num_layers = num_layers,bidirectional = bidirectional,dropout=dropout,device=self.device).to(self.device)


    def train_epoch(self,train_loader, encoder, decoder, encoder_optim,decoder_optim, loss_criterion,teacher_forcing_ratio,ignore_padding=True,device='cpu'):

        """
        Method to train the encoder-decoder model for 1 epoch.

        Params:

            train_loader : The dataloader object (which wraps around WordDataset object of Core_Utils) corresponding to the traindata.
            encoder : The encoder model object.
            decoder : The decoder model object.
            encoder_optim : A torch optim object, corresponding to the optimizer of encoder.
            decoder_optim : A torch optim object, corresponding to the optimizer of decoder.
            loss_criterion : The loss criterion
            teacher_forcing_ratio : The teacher forcing ratio to be used.
            ignore_padding : True, by default.
            device : CPU by default.

        Returns:
            Loss and accuracy of the current epoch.

        """

        tot_correct_word_preds = 0
        tot_words = 0
        epoch_loss = 0

        for data in tqdm(train_loader):

            input_tensor, target_tensor,_,_ = data

            encoder_optim.zero_grad()
            decoder_optim.zero_grad()

            batch_size = data[0].shape[0]

            if encoder.rnn_type == "LSTM":
                encoder_hidden = torch.zeros(encoder.num_layers*encoder.D, batch_size, encoder.hidden_size, device=device)
                encoder_cell = torch.zeros(encoder.num_layers*encoder.D, batch_size, encoder.hidden_size, device=device)
            else:
                encoder_hidden = None
                encoder_cell = None

            encoder_hidden_contexts, encoder_last_hidden, encoder_cell = encoder(input_tensor,encoder_hidden,encoder_cell)

            decoder_outputs, _, _ = decoder(encoder_hidden_contexts, encoder_last_hidden,encoder_cell, target_tensor=target_tensor,teacher_forcing_ratio = teacher_forcing_ratio)

            multi_step_preds = torch.argmax(decoder_outputs,dim=2)
            multi_step_pred_correctness = (multi_step_preds ==  target_tensor)
            num_words = multi_step_preds.shape[0]

            if ignore_padding: ## if padding has to be ignored.

                ## for each word, based on the padding token ID, find the first occurance of the padding token, marking the begining of padding.
                ## argmax is not supported for bool on cuda, hence casting it to long.
                padding_start = torch.argmax((target_tensor == train_loader.dataset.pad_token_id).to(torch.long),dim=1).to(device)
                ## Creating a mask with 1's in each position of a padding token
                mask = (torch.arange(target_tensor.size(1)).unsqueeze(0).to(device) >= padding_start.unsqueeze(1))

                ##doing a logical OR with the mask makes sure that the padding tokens do not affect the correctness of the word
                tot_correct_word_preds += (torch.all(torch.logical_or(multi_step_pred_correctness,mask),dim=1).int().sum()).item()
                tot_words += num_words

            loss = loss_criterion(
                decoder_outputs.view(-1, decoder_outputs.size(-1)),
                target_tensor.view(-1)
            )
            loss.backward()

            encoder_optim.step()
            decoder_optim.step()

            epoch_loss += loss.item()

        epoch_loss = round(epoch_loss / len(train_loader),4)
        epoch_accuracy = round(tot_correct_word_preds*100/tot_words,2)

        return epoch_loss,epoch_accuracy

    def train(self,train_loader,valid_loader, encoder, decoder, epochs,padding_idx,optimiser = "adam",loss="crossentropy",weight_decay=0, lr=0.001,teacher_forcing_ratio = 0,device='cpu',wandb_logging = False):

        """
        The method to train the encoder-decoder model. Makes use of other methods like train_epoch, compute_accuracy to train and return the accuracy.

        train_loader : The dataloader object (which wraps around WordDataset object of Core_Utils) corresponding to the traindata.
        valid_loader : The dataloader object (which wraps around WordDataset object of Core_Utils) corresponding to the Validaiton data.
        encoder : The encoder model object.
        decoder : The decoder model object.
        epochs : Number of epochs of training.
        padding_idx : The index corresponding to the <pad> token.
        optimiser : The optimiser used for training, "adam"/"nadam"/"rmsprop", default : "adam". (Case sensitive)
        loss : The loss function, only "crossentropy" is supported.
        weight_decay : L2, regularization of encoder and decoder model weights.
        lr : The learning rate, default is 0.001
        teacher_forcing_ratio : Teacher forcing ratio, default is 0.
        device : Default is CPU.
        wandb_logging : Default is False.

        """

        ## specify the optimiser
        if optimiser.lower() == "adam":
            encoder_optimizer = optim.Adam(encoder.parameters(), lr=lr,weight_decay=weight_decay)
            decoder_optimizer = optim.Adam(decoder.parameters(), lr=lr,weight_decay=weight_decay)

        elif optimiser.lower() == "nadam":
            encoder_optimizer = optim.NAdam(encoder.parameters(), lr=lr,weight_decay=weight_decay)
            decoder_optimizer = optim.NAdam(decoder.parameters(), lr=lr,weight_decay=weight_decay)

        elif optimiser.lower() == "rmsprop":
            encoder_optimizer = optim.RMSprop(encoder.parameters(), lr=lr,weight_decay=weight_decay)
            decoder_optimizer = optim.RMSprop(decoder.parameters(), lr=lr,weight_decay=weight_decay)

        ## Specify the loss criteria
        if loss.lower() == "crossentropy":
            loss_criterion = nn.CrossEntropyLoss(ignore_index = padding_idx).to(device)

        lp = train_loader.dataset.lp

        for epoch in tqdm(range(epochs)):

            ## Train for 1 epoch.
            train_loss,train_accuracy = self.train_epoch(train_loader, encoder, decoder, encoder_optimizer, decoder_optimizer, loss_criterion,teacher_forcing_ratio,device=device)

            ## compute validation accuracy.
            val_loss,_,val_accuracy = self.compute_accuracy(valid_loader,encoder,decoder,loss_criterion,ignore_padding=True,device=device)

            print(f"Epoch {epoch+1}\t Train Loss : {train_loss}\t Train Acc : {train_accuracy}% \t Val Loss : {val_loss}\t Val Acc : {val_accuracy}%")
            if wandb_logging:
                wandb.log({'epoch': epoch+1,'train loss': train_loss, 'train accuracy': train_accuracy, 'Validation loss': val_loss, 'Validation accuracy': val_accuracy})

    def compute_accuracy(self,dataloader,encoder,decoder,criterion,ignore_padding = True,device='cpu',save_results=False,filename=""):

        """
        Method to compute the accuracy using the model (encoder-decoder) using dataloader.

        This method returns word and character level accuracy.

            Word Level Accuracy : Accuracy is computed at the word level and a word is right iff every character is predicted correctly.
            Char Level Accuracy : Accuracy is computed by comparing each predicted character wrt the correct char.

        Params:

            dataloader : The train/test/valid dataloader.
            encoder : The encoder
            decoder : The decoder
            padding_token_id : The id of the padding token.
            ignore_padding : If True, then in word level accuracy, the padding characters are ignored in computing the word level accuracy.
                            char level accuracy, the padding characters are not considered at all.

                            If false, padding is considered to be a part of the word (for word level accuracy) and the entire word is checked.
            device : Default CPU.
            save_results : Default is False. If true, the results of predictions are saved to a CSV, named by the next param.
            filename : The name of the file, if results have to be saved.

        """

        char_lvl_accuracy = 0
        word_level_accuracy = 0

        tot_chars = 0
        tot_words = 0

        tot_correct_char_preds = 0
        tot_correct_word_preds = 0

        loss = 0

        if save_results:
            rows = []

        #criterion = loss_criterion.to(device)

        with torch.no_grad():

            train = 0

            if encoder.training and decoder.training: ## reset the the model back to train mode
                train = 1

            encoder.eval()
            decoder.eval()

            for data in dataloader:

                input_tensor, target_tensor,_,target_max_len = data

                batch_size = data[0].shape[0]

                if encoder.rnn_type == "LSTM":
                    encoder_hidden = torch.zeros(encoder.num_layers*encoder.D, batch_size, encoder.hidden_size, device=device)
                    encoder_cell = torch.zeros(encoder.num_layers*encoder.D, batch_size, encoder.hidden_size, device=device)
                else:
                    encoder_hidden = None
                    encoder_cell = None


                encoder_hidden_contexts, encoder_last_hidden, encoder_cell = encoder(input_tensor,encoder_hidden,encoder_cell)
                ## even though we are passing target tensor, the teacher forcing ratio is 0, so no teacher forcing
                decoder_outputs, _, attentions = decoder(encoder_hidden_contexts, encoder_last_hidden,encoder_cell, target_tensor = target_tensor,teacher_forcing_ratio = 0)
                loss += criterion(decoder_outputs.view(-1, decoder_outputs.size(-1)), target_tensor.view(-1)).item()

                ## For a batch, for each character find the most probable output word.
                multi_step_preds = torch.argmax(decoder_outputs,dim=2)
                multi_step_pred_correctness = (multi_step_preds ==  target_tensor)
                num_chars = multi_step_preds.numel() ##find the total number of characters in the current batch
                num_words = multi_step_preds.shape[0] ##find the total number of words in the current batch.

                if ignore_padding: ## if padding has to be ignored.

                    ## for each word, based on the padding token ID, find the first occurance of the padding token, marking the begining of padding.

                    ## argmax is not supported for bool on cuda, hence casting it to long.
                    padding_start = torch.argmax((target_tensor == dataloader.dataset.pad_token_id).to(torch.long),dim=1).to(device)
                    ## Creating a mask with 1's in each position of a padding token
                    mask = (torch.arange(target_tensor.size(1)).unsqueeze(0).to(device) >= padding_start.unsqueeze(1))
                    #print(mask)

                    ##doing a logical OR with the mask makes sure that the padding tokens do not affect the correctness of the word
                    tot_correct_word_preds += (torch.all(torch.logical_or(multi_step_pred_correctness,mask),dim=1).int().sum()).item()
                    tot_words += num_words

                    ##creating a complement of the mask so to mark padding tokens as irrelevant
                    complement_mask = (1-mask.int()).bool()
                    num_pad_chars = mask.int().sum().item()
                    ##counting number of non_pad_chars to compute accuracy.
                    num_non_pad_chars = num_chars - num_pad_chars

                    tot_correct_char_preds += (torch.logical_and(multi_step_pred_correctness,complement_mask).int().sum()).item()
                    tot_chars += num_non_pad_chars

                    if save_results:
                        word_preds_correctness = torch.all(torch.logical_or(multi_step_pred_correctness,mask),dim=1).int()
                        for i in range(multi_step_preds.shape[0]):
                            rows.append([dataloader.dataset.lp.decode_word(input_tensor[i].cpu().numpy(),lang_id=0),dataloader.dataset.lp.decode_word(target_tensor[i].cpu().numpy(),lang_id=1),dataloader.dataset.lp.decode_word(multi_step_preds[i].cpu().numpy(),lang_id=1),word_preds_correctness[i].cpu().item()])


                else: ##otherwise.

                    tot_correct_word_preds += (torch.all(multi_step_pred_correctness,dim=1).int().sum()).item()
                    tot_words += num_words

                    tot_correct_char_preds += (multi_step_pred_correctness.int().sum()).item()
                    tot_chars += num_chars

            char_lvl_accuracy = round(tot_correct_char_preds*100/tot_chars,2)
            word_lvl_accuracy = round(tot_correct_word_preds*100/tot_words,2)

            loss /= dataloader.dataset.data.shape[0]

            if save_results:
                df = pd.DataFrame(data=rows, columns=["Source Word","Target Word","Predicted Word","Is Prediction Correct"])
                df.to_csv(filename+".csv",index=False)

            if train:

                encoder.train()
                decoder.train()

            return round(loss,4),char_lvl_accuracy,word_lvl_accuracy

Writing Machine_Translator.py


In [ ]:
%%writefile train_vanilla.py

import numpy as np

import pandas as pd

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import Dataset,TensorDataset, DataLoader, RandomSampler
from torch.nn.utils.rnn import pad_sequence
import random
from functools import partial

import numpy as np

from tqdm.notebook import tqdm

#!python3 -m pip install wandb
import wandb

import argparse

from Core_Utils import *

from Encoder_Decoder_Architecture import *

from Machine_Translator import *

seed = 23

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

def setup_and_start_expt(config,wandb_log = False):

    """
    Method to run experiments as per the specified config and parameters.

    config : A dictionary of paramter-values.
    wandb_log : Default False, if true, the accuracies would be logged.

    Returns:
        Trained MachineTranslator model.
    """

    batch_size = config['batch_size']
    target_lang = "tel"

    base_dir = "data_folder/"

    if config['device'] == None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    else:
        device = torch.device(config['device'])

    use_meta_tokens = True

    lang_dir = base_dir + target_lang + "/"

    ##creating train loader
    train_lp = LanguageProcessor(language_directory=lang_dir,target_lang_name=target_lang,mode="train",meta_tokens=use_meta_tokens)
    ## the ids of these tokens are the same in the source and target language
    pad_token_id = train_lp.source_char2id['<pad>']

    collate_fn_ptr = partial(collate_fn,pad_token_id=pad_token_id,device=device)

    train_dataset = WordDataset(train_lp,device=device)
    train_loader = DataLoader(train_dataset, batch_size=batch_size,collate_fn=collate_fn_ptr, shuffle=True)

    ## creating test loader
    test_lp = LanguageProcessor(language_directory=lang_dir,target_lang_name=target_lang,mode="test",meta_tokens=use_meta_tokens)

    ## to make sure that the same vocabulary and dictionaries are used everywhere
    test_lp.source_char2id = train_lp.source_char2id
    test_lp.source_id2char = train_lp.source_id2char
    test_lp.target_char2id = train_lp.target_char2id
    test_lp.target_id2char = train_lp.target_id2char

    test_dataset = WordDataset(test_lp,device=device)
    test_loader = DataLoader(test_dataset, batch_size=batch_size,collate_fn=collate_fn_ptr, shuffle=True)

    ## creating validation loader
    valid_lp = LanguageProcessor(language_directory=lang_dir,target_lang_name=target_lang,mode="valid",meta_tokens=use_meta_tokens)
    valid_lp.source_char2id = train_lp.source_char2id
    valid_lp.source_id2char = train_lp.source_id2char
    valid_lp.target_char2id = train_lp.target_char2id
    valid_lp.target_id2char = train_lp.target_id2char

    valid_dataset = WordDataset(valid_lp,device=device)
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size,collate_fn=collate_fn_ptr, shuffle=True)

    ##in principle these are all fixed across train/test/valid data

    source_vocab_size = len(list(train_lp.source_char2id.keys()))
    target_vocab_size = len(list(train_lp.target_char2id.keys()))

    hidden_size = config['hidden_size']
    embedding_size = hidden_size

    epochs = config['epochs']

    optimiser = config['optimiser']

    weight_decay = config['weight_decay']

    lr = config['lr']

    num_layers = config['num_layers']

    ## Allowed Values : "GRU"/"RNN"/"LSTM" (not case sensitive)
    rnn_type = config['rnn_type'].upper()

    bidirectional = config['bidirectional']
    teacher_forcing_ratio = config['teacher_forcing_ratio']

    dropout=config['dropout']

    use_attention = config['use_attention']

    model = MachineTranslator(source_vocab_size,target_vocab_size,hidden_size,embedding_size,rnn_type,batch_size,pad_token_id,dropout,num_layers,bidirectional,use_attention,device)

    model.train(train_loader,valid_loader,model.encoder, model.decoder, epochs,padding_idx = pad_token_id,optimiser = optimiser,weight_decay=weight_decay, lr=lr,teacher_forcing_ratio = teacher_forcing_ratio,device=device,wandb_logging = wandb_log)

    loss_criterion = nn.CrossEntropyLoss(ignore_index = pad_token_id).to(device)
    test_loss,_,test_accuracy = model.compute_accuracy(test_loader,model.encoder,model.decoder,loss_criterion,ignore_padding=True,device=device,save_results=True,filename = "predictions_attention")
    print(f"Testing Loss : {test_loss}\tTesting Accuracy : {test_accuracy}%")

    return model

parser = argparse.ArgumentParser()

parser.add_argument("-b", "--batch_size", type=int, default=64,help="Batch size used to train neural network.")

parser.add_argument("-bid", "--bidirectional", type=bool,default=True,choices=[True,False],help="If True, input would be seen in both directions.")

parser.add_argument("-dpt", "--dropout", type=float, default=0.4,help="The dropout probability.")

parser.add_argument("-es", "--embedding_size", type=int, default=256,help="The input embedding dimension.")

parser.add_argument("-e", "--epochs", type=int, default=15,help="Number of epochs to train.")

parser.add_argument("-hs", "--hidden_size", type=int, default=512,help="The dimension of the hidden state.")

parser.add_argument("-lr", "--learning_rate", type=float, default=3e-4,help="Learning rate used to optimize model parameters.")

parser.add_argument("-nl", "--num_layers", type=int, default=4,help="Number of Recurrence Layers.")

parser.add_argument("-o", "--optimizer", type=str, default="nadam",choices=["rmsprop", "adam", "nadam"],help="Optimizer used to minimize the loss.")

parser.add_argument("-rt", "--rnn_type", type=str, default="lstm",choices=["lstm", "gru", "rnn"],help="The type of recurrent cell to be used.")

parser.add_argument("-tf", "--teacher_forcing", type=float, default=0.4,help="The Teacher Forcing Ratio.")

parser.add_argument("-w_d", "--weight_decay", type=float, default=1e-5,help="Weight decay used by optimizers.")

parser.add_argument("-d", "--device", type=str, default=None,help="The device on which the training happens.")

args = parser.parse_args()

config = {

        'batch_size':args.batch_size,

        'bidirectional' :  args.bidirectional,

        'dropout' : args.dropout,

        'embedding_size':args.embedding_size,

        'epochs' : args.epochs,

        'hidden_size':args.hidden_size,

        'lr':  args.learning_rate,

        'num_layers' : args.num_layers,

        'optimiser': args.optimizer,

        'rnn_type' : args.rnn_type,

        'teacher_forcing_ratio' : args.teacher_forcing,

        'weight_decay': args.weight_decay,

        'device' : args.device,

        'use_attention' : False
}

model = setup_and_start_expt(config)

Writing train_vanilla.py


In [ ]:
%%writefile train_attention.py

import numpy as np

import pandas as pd

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import Dataset,TensorDataset, DataLoader, RandomSampler
from torch.nn.utils.rnn import pad_sequence
import random
from functools import partial

import numpy as np
import matplotlib.pyplot as plt
#plt.switch_backend('agg')
import matplotlib.ticker as ticker

from tqdm.notebook import tqdm

#!python3 -m pip install wandb
import wandb

import argparse

from Core_Utils import *

from Encoder_Decoder_Architecture import *

from Machine_Translator import *

seed = 23

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

def setup_and_start_expt(config,wandb_log = False):

    batch_size = config['batch_size']
    target_lang = "tel"

    base_dir = "data_folder/"

    if config['device'] == None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    else:
        device = torch.device(config['device'])

    use_meta_tokens = True

    lang_dir = base_dir + target_lang + "/"

    ##creating train loader
    train_lp = LanguageProcessor(language_directory=lang_dir,target_lang_name=target_lang,mode="train",meta_tokens=use_meta_tokens)
    ## the ids of these tokens are the same in the source and target language
    pad_token_id = train_lp.source_char2id['<pad>']

    collate_fn_ptr = partial(collate_fn,pad_token_id=pad_token_id,device=device)

    train_dataset = WordDataset(train_lp,device=device)
    train_loader = DataLoader(train_dataset, batch_size=batch_size,collate_fn=collate_fn_ptr, shuffle=True)

    ## creating test loader
    test_lp = LanguageProcessor(language_directory=lang_dir,target_lang_name=target_lang,mode="test",meta_tokens=use_meta_tokens)

    ## to make sure that the same vocabulary and dictionaries are used everywhere
    test_lp.source_char2id = train_lp.source_char2id
    test_lp.source_id2char = train_lp.source_id2char
    test_lp.target_char2id = train_lp.target_char2id
    test_lp.target_id2char = train_lp.target_id2char

    test_dataset = WordDataset(test_lp,device=device)
    test_loader = DataLoader(test_dataset, batch_size=batch_size,collate_fn=collate_fn_ptr, shuffle=True)

    ## creating validation loader
    valid_lp = LanguageProcessor(language_directory=lang_dir,target_lang_name=target_lang,mode="valid",meta_tokens=use_meta_tokens)
    valid_lp.source_char2id = train_lp.source_char2id
    valid_lp.source_id2char = train_lp.source_id2char
    valid_lp.target_char2id = train_lp.target_char2id
    valid_lp.target_id2char = train_lp.target_id2char

    valid_dataset = WordDataset(valid_lp,device=device)
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size,collate_fn=collate_fn_ptr, shuffle=True)


    ##in principle these are all fixed across train/test/valid data

    source_vocab_size = len(list(train_lp.source_char2id.keys()))
    target_vocab_size = len(list(train_lp.target_char2id.keys()))

    hidden_size = config['hidden_size']
    embedding_size = hidden_size

    epochs = config['epochs']

    optimiser = config['optimiser']

    weight_decay = config['weight_decay']

    lr = config['lr']

    num_layers = config['num_layers']

    ## Allowed Values : "GRU"/"RNN"/"LSTM" (not case sensitive)
    rnn_type = config['rnn_type'].upper()

    bidirectional = config['bidirectional']
    teacher_forcing_ratio = config['teacher_forcing_ratio']

    dropout=config['dropout']

    use_attention = config['use_attention']

    model = MachineTranslator(source_vocab_size,target_vocab_size,hidden_size,embedding_size,rnn_type,batch_size,pad_token_id,dropout,num_layers,bidirectional,use_attention,device)

    model.train(train_loader,valid_loader,model.encoder, model.decoder, epochs,padding_idx = pad_token_id,optimiser = optimiser,weight_decay=weight_decay, lr=lr,teacher_forcing_ratio = teacher_forcing_ratio,device=device,wandb_logging = wandb_log)

    loss_criterion = nn.CrossEntropyLoss(ignore_index = pad_token_id).to(device)
    test_loss,_,test_accuracy = model.compute_accuracy(test_loader,model.encoder,model.decoder,loss_criterion,ignore_padding=True,device=device,save_results=True,filename = "predictions_attention")
    print(f"Testing Loss : {test_loss}\tTesting Accuracy : {test_accuracy}%")

    return model

parser = argparse.ArgumentParser()

parser.add_argument("-b", "--batch_size", type=int, default=512,help="Batch size used to train neural network.")

parser.add_argument("-bid", "--bidirectional", type=bool,default=True,choices=[True,False],help="If True, input would be seen in both directions.")

parser.add_argument("-dpt", "--dropout", type=float, default=0,help="The dropout probability.")

parser.add_argument("-es", "--embedding_size", type=int, default=256,help="The input embedding dimension.")

parser.add_argument("-e", "--epochs", type=int, default=20,help="Number of epochs to train.")

parser.add_argument("-hs", "--hidden_size", type=int, default=512,help="The dimension of the hidden state.")

parser.add_argument("-lr", "--learning_rate", type=float, default=3e-4,help="Learning rate used to optimize model parameters.")

parser.add_argument("-nl", "--num_layers", type=int, default=2,help="Number of Recurrence Layers.")

parser.add_argument("-o", "--optimizer", type=str, default="rmsprop",choices=["rmsprop", "adam", "nadam"],help="Optimizer used to minimize the loss.")

parser.add_argument("-rt", "--rnn_type", type=str, default="gru",choices=["lstm", "gru", "rnn"],help="The type of recurrent cell to be used.")

parser.add_argument("-tf", "--teacher_forcing", type=float, default=0.4,help="The Teacher Forcing Ratio.")

parser.add_argument("-w_d", "--weight_decay", type=float, default=1e-5,help="Weight decay used by optimizers.")

parser.add_argument("-d", "--device", type=str, default=None,help="The device on which the training happens.")

args = parser.parse_args()

config = {

        'batch_size':args.batch_size,

        'bidirectional' :  args.bidirectional,

        'dropout' : args.dropout,

        'embedding_size':args.embedding_size,

        'epochs' : args.epochs,

        'hidden_size':args.hidden_size,

        'lr':  args.learning_rate,

        'num_layers' : args.num_layers,

        'optimiser': args.optimizer,

        'rnn_type' : args.rnn_type,

        'teacher_forcing_ratio' : args.teacher_forcing,

        'weight_decay': args.weight_decay,

        'device' : args.device,

        'use_attention' : True
}

model = setup_and_start_expt(config)



Writing train_attention.py


In [ ]:
# Create a directory structure compatible with the code
!mkdir -p data_folder/tel

# Copy the Telugu dataset files from Dakshina to the required directory
!cp /content/drive/MyDrive/dakshina_dataset_v1.0/te/lexicons/te.translit.sampled.train.tsv data_folder/tel/train.txt
!cp /content/drive/MyDrive/dakshina_dataset_v1.0/te/lexicons/te.translit.sampled.dev.tsv data_folder/tel/valid.txt
!cp /content/drive/MyDrive/dakshina_dataset_v1.0/te/lexicons/te.translit.sampled.test.tsv data_folder/tel/test.txt


!ls -la data_folder/tel/

total 2564
drwxr-xr-x 2 root root    4096 May 20 00:05 .
drwxr-xr-x 3 root root    4096 May 20 00:05 ..
-rw------- 1 root root  205700 May 20 00:05 test.txt
-rw------- 1 root root 2202728 May 20 00:05 train.txt
-rw------- 1 root root  201819 May 20 00:05 valid.txt


In [ ]:
%%writefile run_sweep.py
import os
import sys
import wandb
import subprocess
import json

def run_sweep(is_attention=False):
    # Initialize wandb
    wandb.login()

    # Define sweep configuration
    sweep_config = {
        'method': 'bayes',
        'name': 's2s_sweep',
        'metric': {
            'name': 'val_acc',
            'goal': 'maximize'
        },
        'parameters': {
            'rnn_type': {
                'values': ['lstm', 'gru', 'rnn']
            },
            'embedding_size': {
                'values': [64, 128, 256]
            },
            'hidden_size': {
                'values': [64, 128, 256]
            },
            'num_layers': {
                'values': [1, 2, 3]
            },
            'dropout': {
                'values': [0.0, 0.2, 0.3]
            },
            'bidirectional': {
                'values': ['True', 'False']
            },
            'optimizer': {
                'values': ['rmsprop', 'adam', 'nadam']
            }
        }
    }

    project_name = "cs6910_assignment3_attention" if is_attention else "cs6910_assignment3_vanilla"
    sweep_id = wandb.sweep(sweep_config, project=project_name)

    # Function to execute when agent is called
    def agent_function():
        with wandb.init() as run:
            # Get configuration from wandb
            config = wandb.config

            # Generate the command
            script = "train_attention.py" if is_attention else "train_vanilla.py"
            cmd = [
                "python",
                script,
                "-b", "64",
                "-bid", config.bidirectional,
                "-dpt", str(config.dropout),
                "-es", str(config.embedding_size),
                "-e", "10",  # Reduced epochs for sweep
                "-hs", str(config.hidden_size),
                "-nl", str(config.num_layers),
                "-o", config.optimizer,
                "-rt", config.rnn_type,
                "-tf", "0.4",
                "-w_d", "1e-5"
            ]

            # Run the command and capture output
            process = subprocess.Popen(
                cmd,
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE,
                universal_newlines=True
            )

            # Process and log output
            for line in process.stdout:
                print(line.strip())
                if "Epoch" in line and "Train Loss" in line and "Val Acc" in line:
                    try:
                        parts = line.split()
                        epoch = int(parts[1])
                        train_loss = float(parts[4])
                        train_acc = float(parts[7].rstrip('%'))
                        val_loss = float(parts[10])
                        val_acc = float(parts[13].rstrip('%'))

                        wandb.log({
                            "epoch": epoch,
                            "train_loss": train_loss,
                            "train_acc": train_acc,
                            "val_loss": val_loss,
                            "val_acc": val_acc
                        })
                    except (ValueError, IndexError) as e:
                        print(f"Error parsing metrics: {e}")

                # Also log test results if present
                if "Testing Loss" in line and "Testing Accuracy" in line:
                    try:
                        parts = line.split()
                        test_loss = float(parts[2])
                        test_acc = float(parts[5].rstrip('%'))
                        wandb.log({
                            "test_loss": test_loss,
                            "test_acc": test_acc
                        })
                    except (ValueError, IndexError) as e:
                        print(f"Error parsing test metrics: {e}")

            # Handle errors
            for line in process.stderr:
                print(f"Error: {line.strip()}")

            process.wait()

    # Start the sweep agent
    wandb.agent(sweep_id, function=agent_function, count=5)  # Reduced count for testing

if __name__ == "__main__":
    # Check command line arguments
    is_attention = False
    if len(sys.argv) > 1 and sys.argv[1] == "attention":
        is_attention = True

    run_sweep(is_attention)

Writing run_sweep.py


In [ ]:
# Write out the run_vanilla_sweep.py file
%%writefile run_vanilla_sweep.py
import wandb
import subprocess
import numpy as np
import random
import torch
import os
import sys
import json

# Set seeds for reproducibility
seed = 23
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

# Define sweep configuration
sweep_config = {
    'method': 'bayes',
    'name': 's2s_vanilla_sweep',
    'metric': {
        'name': 'val_acc',
        'goal': 'maximize'
    },
    'parameters': {
        'cell_type': {
            'values': ['lstm', 'gru', 'rnn']
        },
        'embedding_size': {
            'values': [64, 128, 256]
        },
        'hidden_size': {
            'values': [64, 128, 256]
        },
        'num_layers': {
            'values': [1, 2, 3]
        },
        'dropout': {
            'values': [0.0, 0.2, 0.3]
        },
        'bidirectional': {
            'values': [True, False]
        },
        'optimizer': {
            'values': ['rmsprop', 'adam', 'nadam']
        }
    }
}

# Initialize sweep
sweep_id = wandb.sweep(sweep_config, project="cs6910_assignment3_vanilla")

def run_training():
    # Initialize run
    run = wandb.init()

    # Get hyperparams from wandb
    config = wandb.config

    # Construct command with appropriate arguments
    cmd = [
        'python', 'train_vanilla.py',
        '-b', '64',
        '-bid', str(config.bidirectional),
        '-dpt', str(config.dropout),
        '-es', str(config.embedding_size),
        '-e', '10',  # Reduced epochs for faster sweeps
        '-hs', str(config.hidden_size),
        '-lr', '3e-4',
        '-nl', str(config.num_layers),
        '-o', config.optimizer,
        '-rt', config.cell_type,
        '-tf', '0.4',
        '-w_d', '1e-5'
    ]

    # Run the command and capture output
    process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)

    # Initialize metrics to track
    val_acc = 0
    train_acc = 0
    val_loss = 0
    train_loss = 0

    # Process output line by line
    for line in iter(process.stdout.readline, ""):
        print(line.strip())

        # Extract metrics from training output
        if "Val Acc :" in line:
            parts = line.split()
            epoch_idx = parts.index("Epoch")
            train_loss_idx = parts.index("Train") + 2
            train_acc_idx = parts.index("Train") + 5
            val_loss_idx = parts.index("Val") + 2
            val_acc_idx = parts.index("Val") + 5

            epoch = int(parts[epoch_idx + 1])
            train_loss = float(parts[train_loss_idx])
            train_acc = float(parts[train_acc_idx].replace("%", ""))
            val_loss = float(parts[val_loss_idx])
            val_acc = float(parts[val_acc_idx].replace("%", ""))

            # Log metrics
            wandb.log({
                "epoch": epoch,
                "train_loss": train_loss,
                "train_acc": train_acc,
                "val_loss": val_loss,
                "val_acc": val_acc
            })

    # Wait for process to complete
    process.wait()

    # Log final metrics
    wandb.log({
        "final_val_acc": val_acc,
        "final_train_acc": train_acc,
        "final_val_loss": val_loss,
        "final_train_loss": train_loss
    })

# Start sweep
wandb.agent(sweep_id, function=run_training, count=10)

Writing run_vanilla_sweep.py


In [ ]:
# Write out the run_attention_sweep.py file
%%writefile run_attention_sweep.py
import wandb
import subprocess
import numpy as np
import random
import torch
import os
import sys
import json

# Set seeds for reproducibility
seed = 23
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

# Define sweep configuration
sweep_config = {
    'method': 'bayes',
    'name': 's2s_attention_sweep',
    'metric': {
        'name': 'val_acc',
        'goal': 'maximize'
    },
    'parameters': {
        'cell_type': {
            'values': ['lstm', 'gru', 'rnn']
        },
        'embedding_size': {
            'values': [64, 128, 256]
        },
        'hidden_size': {
            'values': [64, 128, 256]
        },
        'num_layers': {
            'values': [1, 2, 3]
        },
        'dropout': {
            'values': [0.0, 0.2, 0.3]
        },
        'bidirectional': {
            'values': [True, False]
        },
        'optimizer': {
            'values': ['rmsprop', 'adam', 'nadam']
        }
    }
}

# Initialize sweep
sweep_id = wandb.sweep(sweep_config, project="cs6910_assignment3_attention")

def run_training():
    # Initialize run
    run = wandb.init()

    # Get hyperparams from wandb
    config = wandb.config

    # Construct command with appropriate arguments
    cmd = [
        'python', 'train_attention.py',
        '-b', '64',
        '-bid', str(config.bidirectional),
        '-dpt', str(config.dropout),
        '-es', str(config.embedding_size),
        '-e', '10',  # Reduced epochs for faster sweeps
        '-hs', str(config.hidden_size),
        '-lr', '3e-4',
        '-nl', str(config.num_layers),
        '-o', config.optimizer,
        '-rt', config.cell_type,
        '-tf', '0.4',
        '-w_d', '1e-5'
    ]

    # Run the command and capture output
    process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)

    # Initialize metrics to track
    val_acc = 0
    train_acc = 0
    val_loss = 0
    train_loss = 0

    # Process output line by line
    for line in iter(process.stdout.readline, ""):
        print(line.strip())

        # Extract metrics from training output
        if "Val Acc :" in line:
            parts = line.split()
            epoch_idx = parts.index("Epoch")
            train_loss_idx = parts.index("Train") + 2
            train_acc_idx = parts.index("Train") + 5
            val_loss_idx = parts.index("Val") + 2
            val_acc_idx = parts.index("Val") + 5

            epoch = int(parts[epoch_idx + 1])
            train_loss = float(parts[train_loss_idx])
            train_acc = float(parts[train_acc_idx].replace("%", ""))
            val_loss = float(parts[val_loss_idx])
            val_acc = float(parts[val_acc_idx].replace("%", ""))

            # Log metrics
            wandb.log({
                "epoch": epoch,
                "train_loss": train_loss,
                "train_acc": train_acc,
                "val_loss": val_loss,
                "val_acc": val_acc
            })

    # Wait for process to complete
    process.wait()

    # Log final metrics
    wandb.log({
        "final_val_acc": val_acc,
        "final_train_acc": train_acc,
        "final_val_loss": val_loss,
        "final_train_loss": train_loss
    })

# Start sweep
wandb.agent(sweep_id, function=run_training, count=10)

Writing run_attention_sweep.py


In [ ]:
# Initialize WandB sweep
import vanilla_sweep_config
import wandb

# Create the sweep
sweep_id = wandb.sweep(vanilla_sweep_config.sweep_config, project="cs6910_assignment3_vanilla")

# Define the sweep agent function
def sweep_agent():
    import train_vanilla
    # Assuming train_vanilla.py has a main() function that accepts command line args
    # and has wandb integration

# Start the sweep
wandb.agent(sweep_id, function=sweep_agent, count=10)  # Adjust count based on how many runs you want

ModuleNotFoundError: No module named 'vanilla_sweep_config'

In [ ]:

!python train_vanilla.py -b 64 -bid True -dpt 0.2 -es 128 -e 15 -hs 256 -lr 3e-4 -nl 2 -o nadam -rt lstm -tf 0.4 -w_d 1e-5
# Run the attention model directly (no sweep)
!python train_attention.py -b 512 -bid True -dpt 0.0 -es 256 -e 15 -hs 512 -lr 3e-4 -nl 2 -o rmsprop -rt gru -tf 0.4 -w_d 1e-5
# Run the vanilla sweep
!python run_vanilla_sweep.py
# Run the attention sweep
!python run_attention_sweep.py

Streaming output truncated to the last 5000 lines.
 64% 588/915 [00:37<00:17, 18.44it/s]
 64% 590/915 [00:37<00:17, 18.39it/s]
 65% 592/915 [00:37<00:17, 18.41it/s]
 65% 594/915 [00:37<00:17, 18.33it/s]
 65% 596/915 [00:38<00:17, 18.61it/s]
 65% 598/915 [00:38<00:16, 18.93it/s]
 66% 600/915 [00:38<00:17, 18.15it/s]
 66% 602/915 [00:38<00:17, 17.55it/s]
 66% 604/915 [00:38<00:17, 17.87it/s]
 66% 606/915 [00:38<00:17, 17.41it/s]
 66% 608/915 [00:38<00:17, 17.45it/s]
 67% 610/915 [00:38<00:17, 17.07it/s]
 67% 612/915 [00:38<00:17, 17.37it/s]
 67% 614/915 [00:39<00:17, 17.17it/s]
 67% 616/915 [00:39<00:16, 17.64it/s]
 68% 618/915 [00:39<00:16, 17.97it/s]
 68% 620/915 [00:39<00:16, 17.37it/s]
 68% 622/915 [00:39<00:16, 17.52it/s]
 68% 624/915 [00:39<00:16, 18.09it/s]
 69% 627/915 [00:39<00:15, 18.37it/s]
 69% 629/915 [00:39<00:15, 18.69it/s]
 69% 631/915 [00:40<00:15, 18.63it/s]
 69% 633/915 [00:40<00:15, 18.31it/s]
 69% 635/915 [00:40<00:15, 17.62it/s]
 70% 637/915 [00:40<00:15, 17.71it/s]

In [ ]:
# Code to generate attention heatmaps from the best attention model
import matplotlib.pyplot as plt
import numpy as np
import torch
from Machine_Translator import MachineTranslator

# Load your best attention model
# Plot attention heatmaps in a 3x3 grid
fig, axes = plt.subplots(3, 3, figsize=(15, 15))
axes = axes.flatten()

# Get sample test examples
# Plot heatmaps for each
for i in range(9):
    # Code to plot attention heatmap on axes[i]
    pass

plt.tight_layout()
plt.savefig('attention_heatmaps.png')
plt.show()